## Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import scipy.io
import itertools as it
import scipy.special as psi
plt.style.use('classic')
import seaborn as sns
import pandas as pd
import math as mt
import time

from scipy.io import loadmat
from scipy import stats
from numpy.random import seed
from numpy.random import rand
from scipy.integrate import quad
from scipy.io import savemat
from tempfile import TemporaryFile
from scipy.io import loadmat
from sklearn.decomposition import PCA
from sklearn.manifold import MDS
from sklearn.decomposition import KernelPCA
from mpl_toolkits import mplot3d
# from mPE_fn import mPE
from scipy.spatial import distance
from scipy.stats import entropy
from mPE_ultis import integrand, ubble, array_list, permutation

## Multiscale Permutation Entropy (mPE)

In [ ]:
# [H,h]=mPE(data,n)
# returns the permutation entropy (of order n) and normalised 
# entropy vector of a time series "data"
    
# [H,h]=mPE(data,n,eps)
# returns the permutation entropy (of order n) and normalised 
# entropy vector of a time series "data". The vector eps contains
# as many elements as the number of scales considered by the algorithm.
# Each element of the vector eps represents how many elements are 
# considered in creating a coarse-grained time series. Default is eps=[1]
    
# [H,h]=mPE(data,n,eps,L)
# returns the permutation entropy (of order n) and normalised 
# entropy vector of a time series "data". The vector eps contains
# as many elements as the number of scales considered by the algorithm.
# Each element of the vector eps represents how many elements are 
# considered in creating a coarse-grained time series. Default is eps=[1]
# L is a lag parameter, default is 1.
# 
# n(order of the entropy)
# L(distance between two adjacent elements in the computation)(generally=1)

def mPE_(*argv):    
    g=len(argv)
    
    if g==2:
        n=argv[1]
        datain=argv[0]
        lamin=np.array([1])
        epsin=np.array([1])
    if g==3:
        n=argv[1]
        datain=argv[0]
        lamin=np.array([1])
        epsin=argv[2]
    if g==4:
        n=argv[1]
        datain=argv[0]
        lamin=argv[3]
        epsin=argv[2]
    
#    Check for the data to be in the right dimension
    if isinstance(datain, list):
        if len(datain[:,1])>len(datain[1,:]):
            datain=datain.transpose()
    else:
        print('check')
        if len(datain[:,0])>len(datain[0,:]):
            datain=datain.transpose()
        
    scalesno=len(epsin)
    lagno=len(lamin)
    HH=np.zeros((lagno,scalesno))
    norm_HH=np.zeros((lagno,scalesno))
#    Definition of parameters: fac is the number of possible permutations
#    Elem is the No of data points
#    Dim is the dimensionality of the samples
    
    for lam in range(0,lagno):
        for eps in range(0,scalesno):
            scale=epsin[eps]
            L=lamin[lam]
            xlen=len(datain[0,:])
            ylen=np.round((xlen/scale)-0.5)
            ylen=ylen.astype(int)
            D=np.zeros((len(datain[:,0]),ylen))
            for ylenc in range(0,ylen):
                dfg=datain[:,((ylenc)*scale):((ylenc+1)*scale)]
                fhk=np.sum(dfg,axis=1)
                r=(1/scale)*fhk
                D[:,ylenc]=r
                
            data=D
            fac=mt.factorial(n)
            elem=len(data[0,:])
            dim=len(data[:,0])
            
        #    A is a n-by-factorial(n) matrix that shows all the 
        #    possible permutations of n elements
            A=permutation(n)
            
        #    counter is a factorial(n) square matrix that counts the recurrence of
        #    a dim-dimensional permutation
            lpi=fac*np.ones((dim))
            lpi=lpi.astype(int)
            nj = lpi.tolist()
            counter=np.zeros((nj))
            
        #    For each iteration i, a series of n points is sampled from the 
        #    data set and the corresponding permutation is identified. 
        #    The counter matrix keeps track of the numiber of times a certain
        #    permutation, or combination of permutations is observed. 
            for i in range(0, elem-n*L+L-1):
                coord=np.zeros((dim))
                for num in range(0,dim):
                    sample=data[num,range(i,i+L*(n),L)]
                    ord=ubble(sample)
                    perm_num=0
                    check_1=1
                    check_2=1
                    
                    while (perm_num<=fac) and (check_2):
                       
                        check_1=1
                        for j in range(0,n-1):
                            if ord[j]!=A[perm_num,j]:
                                check_1=0
                        if check_1:
                            coord[num]=perm_num
                            check_2=0
                            
                        perm_num=perm_num+1
                
                    
                coord=coord.astype(int)
                coord1=tuple(coord)
                counter[coord1]=counter[coord1]+1
                
        #    Once the counter matrix is complete, each element is divided by the
        #    total number of samples to get a empirical probability, and the PE is
        #    computed according to the method described in Schurmann
            
            H=0
            counter1=counter.flatten()
            for iter in range(0,(fac**dim)):
               
                rec=counter1[iter]
                tot=elem-n+1
                
                if rec==0:
                    H=H
                    
                else:
                    I = quad(integrand, 0, 1, args=(rec))
                    I=I[0]
                    coeff=((-1)**rec)*I
                    prob=(rec/tot)*(psi.digamma(tot)-psi.digamma(rec)-coeff)
                    H=H+mt.log2(mt.exp(prob))
                 
        #     The normalised entropy norm_H is computed as well
            norm_H=H/(mt.log2(fac**dim))
            
            HH[lam,eps]=H
            norm_HH[lam,eps]=norm_H
    return [HH,norm_HH]

## Convergence Analysis - Synthetic Data

In [ ]:
# params
orders = [3, 4]
lengths = [6, 8, 10]
sizes = [100, 500, 1000, 5000, 10000, 50000, 100000, 500000, 1000000, 5000000]
bound_len = 1e07
n_trials = 20
# convergence analysis over 20 trials, 3 dims and different lengths
for n_PC in range(2,4):
    for i, order in enumerate(orders):
        for length_ in lengths:
            # initialize sample entropy array
            sample_H = np.zeros((n_trials, len(sizes)))
            for trial in range(n_trials):
                # create fundamental unit for synthetic data
                f_unit = np.random.rand(n_PC, length_)
                
                # calculating entropy bound using the entire trajectory length
                rand_traj = np.zeros((n_PC, int(bound_len)))
                for ii in range(int(bound_len/length_)):
                    
                    # create trajectory for entropy bound
                    idx = ii * length_
                    rand_traj[:, idx:idx+length_] = f_unit
                                
                    # update fundamental unit 
                    f_unit = np.random.rand(n_PC, length_)
            
                print(rand_traj.shape)
                
                #######################################################################
                print("CALCULATING ENTROPY BOUND")
                [H_bound, _] = mPE_(rand_traj, order)
                print("finished")
                #######################################################################
                
                f_unit = np.random.rand(n_PC, length_)
                for j, size in enumerate(sizes):
                    sample_traj = np.zeros((n_PC, size))
                    for iii in range(int(size/length_)):
                                
                        # create trajectory for entropy bound
                        idx = iii * length_
                        sample_traj[:, idx:idx+length_] = f_unit
                                
                        # update fundamental unit 
                        f_unit = np.random.rand(n_PC, length_)

                        
                    #######################################################################
                    print("CALCULATING SAMPLE ENTROPY")
                    [H_sample, _] = mPE_(sample_traj, order)
                    sample_H[trial,j] = H_sample
                    #######################################################################
                
                plt.plot(sizes, sample_H[trial,:])
                plt.axhline(y=H_bound, color="black", linestyle="--")
                plt.ylabel('order ' + str(order) + ' mPE')
                plt.xlabel('sample size')
                plt.xscale("log")
                plt.ylim([np.min(sample_H[i,:]) - 1 , H_bound + 0.5])
                plt.title(str(n_PC) + ' Principal Components')
                plt.grid()
                plt.show()

## Convergence Analysis - Observation Data

## Load Data

In [ ]:
# Get optimal time lag
filename = '/rds/general/user/lr4617/home/4th_Year_Project/CAPTURE_rat_multidimensional/raw_data/normal/traj_1/na_auto_MI_per_lag_6.npy'
sub_sampling = np.load(filename)
step = 5
shifts = step*np.arange(sub_sampling.shape[0])
plt.scatter(shifts, sub_sampling/np.max(sub_sampling))

auto_mi = 0.6*np.max(sub_sampling)
time_lags = np.where(np.isclose(sub_sampling, auto_mi, rtol=1e-02, atol=1e-02))
time_lag = shifts[time_lags[0][0]]

# load entire high-dimensional trajectory
path = '/rds/general/user/lr4617/home/4th_Year_Project/CAPTURE_rat_multidimensional/raw_data/normal/traj_1/trajectories_na/'
trajectories = os.listdir(path)

## Data Cleansing

In [ ]:
# removing invalid values (e.g. NaN)
# input data is already normalized (z-score) but needs to get rid of non-valued datapoints
nan_cols = []
for i, time_bin in enumerate(trajectories):
    if time_bin != 'behavs':
        trajectory = loadmat(path + time_bin)
        trajectory = trajectory['trajectory'] 
        for i in range(trajectory.shape[1]):
            if np.isnan(trajectory[:, i]).all():
                nan_cols.append(i)

sampling_factor = time_lag
nan_cols = np.asarray(nan_cols)
a = 0

if nan_cols.size > 0:
    if len(np.where(nan_cols==nan_cols[0])[0])*3 == len(nan_cols):
        sampled_trajectories = np.zeros(((trajectory.shape[0]*int(len(trajectories)/sampling_factor)), trajectory.shape[1]-len(nan_cols)))
        all_trajectories = np.zeros((trajectory.shape[0]*int(len(trajectories)), trajectory.shape[1]-len(nan_cols)))
else:
    sampled_trajectories = np.zeros(((trajectory.shape[0]*int(len(trajectories)/sampling_factor)), trajectory.shape[1]))
    all_trajectories = np.zeros((trajectory.shape[0]*int(len(trajectories)), trajectory.shape[1]))

for i, time_bin in enumerate(trajectories):
    if time_bin != 'behavs':
        trajectory = loadmat(path + time_bin)
        trajectory = trajectory['trajectory'] 
        if nan_cols.size > 0:
            trajectory = np.delete(trajectory, nan_cols, 1)
        idx = a*trajectory.shape[0]
        idx_2 = i*trajectory.shape[0]
        all_trajectories[idx_2:idx_2+trajectory.shape[0], 0:sampled_trajectories.shape[1]] = trajectory
        if i % sampling_factor == 0 and sampled_trajectories.shape[0]-idx >= trajectory.shape[0]:
            sampled_trajectories[idx:idx+trajectory.shape[0], 0:sampled_trajectories.shape[1]] = trajectory
            a+=1
            
# convert nan to number when not it is a sparse recurrence (not an entire column)
sampled_trajectories = np.nan_to_num(sampled_trajectories)
all_trajectories = np.nan_to_num(all_trajectories)

print(sampled_trajectories.shape)
print(all_trajectories.shape)

In [ ]:
# reduce data according to explained variance values using linear PCA
for trial in range(20):
    for n_PC in range(1,4):
        pca = PCA(n_components=n_PC)
        reduced_traj = pca.fit_transform(sampled_trajectories)
        reduced_traj = np.reshape(reduced_traj, (reduced_traj.shape[0], n_PC))

        # sample size effect on dynamical entropy bound
        min_portion = 20
        orders = [3, 4]
        min_length = int(len(reduced_traj)/min_portion)
        sizes = np.arange(min_length, len(reduced_traj), 2*min_length)
        sample_H = np.zeros((len(orders), sizes.size))

        for i, order in enumerate(orders):
            # calculating entropy bound using the entire trajectory length
            if random:
                rand_uniform_traj = np.random.rand(len(reduced_traj))
                rand_uniform_traj = np.reshape(rand_uniform_traj, (len(reduced_traj), n_PC))
                [H_bound, _] = mPE_(rand_uniform_traj, order)
            else:
                [H_bound, _] = mPE_(reduced_traj, order)

            for j, size in enumerate(sizes):
                if random:
                    valid = False
                    while valid==False:
                        start_idx = np.random.randint(0, high=len(reduced_traj))
                        if start_idx + size <= len(reduced_traj):
                            valid = True
                    traj = reduced_traj[start_idx:start_idx + size, :]
                else:
                    traj = reduced_traj[0:size, :]
                [H_sample, _] = mPE_(traj, order)
                sample_H[i,j] = H_sample

            plt.plot(sizes, sample_H[i,:])
            plt.axhline(y=H_bound, color="black", linestyle="--")
            plt.ylabel('order ' + str(order) + ' mPE')
            plt.xlabel('sample size')
            plt.xscale("log")
            plt.ylim([np.min(sample_H[i,:]) - 1 , H_bound + 0.5])
            plt.title(str(n_PC) + ' Principal Components')
            plt.grid()
            plt.show()